# Approx Lab

The goal of this lab is to gain experience using approximation algorithms.

For the purpose of this assignment, you will be implementing the knapsack problem. Before continuing, the following cell contains any dependencies you may need for this assignment.

In [16]:
import csv

## Background Info

### Knapsack Problem
We are actually going to be looking at a specific (hopefully fun) implementation of the knapsack problem.

The traditional knapsack problem asks you to pack a set of items, with given weights and values into a "knapsack" with a maximum capacity. You cannot pack more items than the capacity of the knapsack. The goal is to select a subset of items that maximize the total value with what can fit in the knapsack. The name "knapsack" comes from the similarity to a hiker trying to pack a bag without going over a certain weight.

The knapsack problem is an NP-Hard optimization problem, which is why it is one of the classic approximation algorithms.

### Specifications for this Lab
The variation that we will be doing in this assignment is referred to as an 0/1 knapsack problem, meaning that items must be completely put in the knapsack or they must be left behind (you can't have a fractional portion of an item). We will be representing this as a simple fantasy sports league - specifically Formula 1 drivers. 

You are provided with a list of F1 drivers and constructors with given costs and points as well as a budget of $100M. You will be selecting five drivers and two constructors (teams). 

### Scoring Information
Points are gained from qualification results for drivers and constructors as well as race results for drivers and constructors. 

<!-- Qualifications
- Drivers score based on their resulting position 
- Constructor score the combined total of their two drivers in qualifying plus a modifier based on how far both their drivers get. -->
<!-- 
| Drivers    | Points |
| -------- | ------- |
| Pole Position (1st Place) | 10 |
| 2nd Place | 9 |
| 3rd Place | 8 |
| 4th Place | 7 |
| 5th Place | 6 |
| 6th Place | 5 |
| 7th Place | 4 |
| 8th Place | 3 |
| 9th Place | 2 |
| 10th Place | 1 |
| 11th-20th Place | 0 |
| No Time Set | 0 |
| Disqualification | 0 | -->


<!-- | Constructors | Points |
| ----------- | ------- |
| Neither Driver Reaches Q2 | -1 |
| One Driver Reaches Q2 | 1 |
| Both Drivers Reach Q2 | 5 |
| Both Drivers Reach Q3 | 10 | -->


<!-- Race
- Drivers score based on their resulting position, position difference from starting order, and some additional bonuses
- Constructors score a combined total of their two drivers' results (excluding the Driver of the Day bonus) ~~plus a modifier based on pitstop times~~ -->

<!-- | Drivers | Points |
| -------- | ------- |
| Positions Gained | 1/position |
| Positions Lost | -1/position |
| Overtakes Made | 1/overtake |
| Fastest Lap | 10 |
| Driver of the Day | 10 | -->

<!-- | Race Result | Points |
| -------- | ------- |
| 1st Place | 25 |
| 2nd Place | 18 |
| 3rd Place | 15 |
| 4th Place | 12 |
| 5th Place | 10 |
| 6th Place | 8 |
| 7th Place | 6 |
| 8th Place | 4 |
| 9th Place | 2 |
| 10th Place | 1 |
| 11th-20th Place | 0 |
| DNF | -20 |
| Disqualification | -20 | -->

Fortunately for you, we have access to the current point standings post-2025 China GP for drivers & constructors, which are available in `drivers.csv` & `constructors.csv`.

*Note: The costs are in millions, so 30 is actually 30 M.

### Questions (Answered in Various Parts of the Lab): 
1. What is the general setup of the problem?
2. Determining the mathematical functions of the problem
3. Implementing a greedy algorithm


## A. Assessing the Problem

The values that are provided for this lab are from the points distribution PRIOR to the Japan Grand Prix (which means we can actually compare with the race results at the submission deadline!)

Create a fantasy team https://fantasy.formula1.com/en/create-team for the upcoming Grand Prix to get a better idea of what drivers, constructors, points, and the budget are. Who is on your team & how much did you spend? Don't spend too much time on this, it really is just to get familiar with the format.

**Leclerc, Bearman, Hulkenberg, Ocon, Tsunoda. Red bull & Haas are the constructors. I spent 29.9 M**

What represents the "items" in the knapsack? 

**The items are each of the drivers and constructors**

What represents the weights?

**The price of each driver/constructor**

What represents the values?

**The points of each driver/constructor (idk whether it is whole season or up to a specific point tho)**

In [17]:
# Load the data from the CSV files into a format that you are comfortable working with
# This can be arrays, dictionaries, or pd DataFrames
constructors_info = {}
driver_info = {}
with open("constructors.csv", 'r') as file:
    reader = csv.reader(file)
    for i, row in enumerate(reader):
        if i != 0:
            constructors_info[row[0]] = [row[1],row[2]]
with open("drivers.csv", 'r') as file:
    reader = csv.reader(file)
    for i, row in enumerate(reader):
        if i != 0:
            driver_info[row[0]] = [row[1],row[2]]

# Print/Display your data as well to confirm that you have successfully saved it.
print(constructors_info)
print(driver_info)

{'McLaren': ['172', '30.6'], 'Mercedes': ['136', '23.3'], 'Red Bull Racing': ['86', '25.4'], 'Haas': ['61', '8.2'], 'Williams': ['36', '13.5'], 'Racing Bulls': ['32', '8.0'], 'Kick Sauber': ['15', '6.2'], 'Ferrari': ['13', '27.1'], 'Aston Martin': ['5', '7.3'], 'Alpine': ['-20', '8.3']}
{'Lando Norris': ['100', '29.6'], 'Andrea Kimi Antonelli': ['61', '19.3'], 'George Russell': ['60', '21.4'], 'Max Verstappen': ['59', '28.6'], 'Oscar Piastri': ['55', '23.0'], 'Lance Stroll': ['33', '9.3'], 'Esteban Ocon': ['32', '8.1'], 'Alexaner Albon': ['28', '12.8'], 'Nico Hulkenberg': ['22', '7.6'], 'Oliver Bearman': ['22', '6.7'], 'Lewis Hamilton': ['3', '23.6'], 'Yuki Tsunoda': ['0', '16.8'], 'Liam Lawson': ['0', '8.4'], 'Charles LeClerc': ['0', '25.3'], 'Pierre Gasly': ['-7', '10.6'], 'Isack Hadjar': ['-9', '5.0'], 'Jack Doohan': ['-9', '6.0'], 'Carlos Sainz': ['-11', '11.9'], 'Gabriel Bortoleto': ['-11', '4.8'], 'Fernando Alonso': ['-36', '7.6']}


# B. Integer Program

Before implementing our approx algorithm, we need to first consider the integer program. 

Given the following variables, create an integer program that maximizes the total points of drivers while remaining under the budget: 
- $x_i$ is the decision variable that determines whether driver $i$ is included in the team or not
- $p_i$ is the # of points that driver $i$ is worth
- $c_i$ is the cost of driver $i$

For now, we will focus on just the drivers and ignore the constructors. We will also temporarily reduce our budget to **$50M**

While we are not planning on implementing the full integer program, it helps us quantify parts of the problem


$\text{maximize} \quad \sum_{i=1}^{5} x_i p_i$

$\text{subject to} \quad \sum_{i=1}^5 x_i c_i \leq 50$

$\quad \quad \quad \quad \quad \quad x_i \in (0,1)$ 




## Decision Variable & Constraints

We can make helper functions for our future algorithm based on constraints that we derive when forming the IP: 
1. Which drivers should be included in our solution? 
2. What is our maximum cost?
3. What is our maximum # of drivers? 

In [18]:
budget = 50 # in millions

#which drivers should be included in our solution?
#what is our max cost?
def under_max_cost(drivers, driver_info=driver_info):
    """
    Takes in the picked drivers, and outputs a bool determining whether 
    it satisfies the first constraint, if it is under the maximum cost

    args:
        drivers: list of strings of the drivers picked in the solution
        driver_info: dict, keys:str, vals: list of str: all the driver info from the csv
    returns:
        bool depending on whether the constraint is satisfied
    """
    cum_cost = 0.0 #cumulitive cost of all the drivers
    for driver in drivers:
        # print(driver_info[driver][1])
        cum_cost += float(driver_info[driver][1])
        # print(f"{cum_cost=}")
    return cum_cost <= budget

# uncomment for testing:
#print(under_max_cost(["Lando Norris", "Nico Hulkenberg", "Jack Doohan", "Andrea Kimi Antonelli"]))

#what is our max num of drivers?
def check_driver_count(drivers):
    """
    Checks the selected list of driver to make sure that there are 5 total

    args:
        drivers: list of strings of the drivers
    returns:
        bool depending on if the total num of drivers is 5
    """
    return len(drivers) == 5

# C. Greedy Algorithm


There are multiple ways we can go about solving the knapsack problem. For this lab, we will be focusing on a greedy algorithm and a variant of that.

Let's start with a basic greedy algorithm:
1. Calculate the value-to-cost ratio for each driver 
2. Sort drivers in non-increasing order of value-to-cost
2. Greedily select drivers from list until either budget or max drivers constraint is hit

Use the helper methods from the previous part in your implementation.

In [48]:
budget = 50
def value_to_cost(driver_info=driver_info): 
    """
    calculates the ratio of value to cost for each driver

    args:
        driver_info: dict, keys:str, vals: list of str: all the driver info from the csv
    returns:
        ratios: dict, keys:str, val:float : each driver mapped to their ratio
    """
    ratios = {}
    for key, val in driver_info.items():
        ratios[key] = float(val[0]) / float(val[1])
    return ratios

def sort_items(info=driver_info): 
    """
    sorts the drivers in order of their value to cost ratio

    args:
        driver_info: dict, keys:str, vals: list of str: all the driver info from the csv
        decreasu
    returns:
        sorted_ratios: dict, keys:str, val:float : each driver mapped to their ratio, in order of most to least ratio
    """
    ratios = value_to_cost(driver_info)
    sorted_ratios = {k: v for k, v in sorted(ratios.items(), key=lambda x:x[1])}
    return sorted_ratios

def greedy_driver_select(driver_info=driver_info):
    """
    Greedily selects which drivers to use based on their value cost ratio

    args:
        driver_info: dict, keys:str, vals: list of str: all the driver info from the csv
    returns:
        selected_drivers: list of drivers selected by the greedy algorithm
    """
    selected_drivers = []
    drivers_sorted = sort_items(driver_info)
    total_cost = 0.0

    for driver in drivers_sorted:
        next_drivers = []
        next_drivers = selected_drivers.copy()
        next_drivers.append(driver)
        if total_cost + float(driver_info[driver][1]) <= budget:
            print(total_cost + float(driver_info[driver][1]))
            print(driver)
            selected_drivers.append(driver)

            total_cost += float(driver_info[driver][1])
    # while under_max_cost(selected_drivers, driver_info):
    #     selected_drivers.append(list(drivers_sorted.keys())[i])
    #     i+=1
    # #needs an extra step to check if the last added driver makes us go over the limit
    # if not under_max_cost(selected_drivers):
    #     selected_drivers.pop()
    return selected_drivers


# testing my code yippee
greedy_drivers = greedy_driver_select()
print(f"selected drivers using greedy algorithm = {greedy_drivers}")
print(f"is greedy actually under max cost? {under_max_cost(greedy_drivers)}")




8.1
Esteban Ocon
17.4
Lance Stroll
47.0
Lando Norris
selected drivers using greedy algorithm = ['Esteban Ocon', 'Lance Stroll', 'Lando Norris']
is greedy actually under max cost? True


Try running your basic greedy algorithm. 

You may notice a few flaws:
1. The greedy algorithm is only looking at value-to-cost ratio, which may miss out on an overall improvement in value that still fits within the budget
2. Depending on your implementation, you may not currently be meeting the "5 total drivers" constraint - this is fine


## Redux Greedy Algorithm
We are going to next implement a small adjustment to this algorithm in order to improve its performance. \
This is a 2-approximation for the knapsack problem. 

1. Calculate the value-to-cost ratio for each driver
2. Sort drivers in non-increasing order of value-to-cost
3. Greedily add items until we hit an item that is too big
4. Compare the too big item against the already existing list of items & pick the better of the two

It might help to make a separate function to handle #4. 

In [50]:
def redux_greedy_driver_select(driver_info=driver_info):
    """
    Performs a redux greedy algorithm on all the drivers to pick
    which ones are optimal for the picks

    args:
        driver_info: dict, keys:str, vals: list of str: all the driver info from the csv
    returns:
        selected drivers: list strings of drivers picked
    """
    sorted_drivers = sort_items(driver_info)
    selected_drivers = []
    # print("next driver cost too large, start reducing and comparing")
    for driver in sorted_drivers.keys():
        next_drivers = []
        # print(selected_drivers)
        # print(under_max_cost(selected_drivers))
        next_drivers = selected_drivers.copy()
        next_drivers.append(driver)
        if not under_max_cost(next_drivers):
            if driver not in selected_drivers:
                selected_drivers = compare_drivers(driver, selected_drivers)         
        else:
            if driver not in selected_drivers:
                selected_drivers.append(driver)
        

            
    return selected_drivers



def compare_drivers(driver1, rest_of_drivers, driver_info=driver_info):
    """
    Decides which of the 2 drivers inputted are better to be picked. 

    args:
        driver1: string of the first driver
        driver2: string of the second driver
        driver_info: dict, keys:str, vals: list of str: all the driver info from the csv
    returns:
        better_driver: list of strings of the better of the 2 options
    """
    ratios = value_to_cost(driver_info)
    d1_ratio = ratios[driver1]
    rest_ratio = 0.0
    for driver in rest_of_drivers:
        rest_ratio += ratios[driver]
    rest_ratio = rest_ratio / len(rest_of_drivers)
    if rest_ratio >= d1_ratio:
        return rest_of_drivers
    else:
        return [driver1]
    
best_drivers = redux_greedy_driver_select()
print(best_drivers)
print(under_max_cost(best_drivers))

    


['Esteban Ocon', 'Lance Stroll', 'Lando Norris']
True


# D. Reintroducing Constructors

We've been neglecting our constructors so far and cutting our budget. Let's make them feel included in our processes again. 

## Integer Program

We should start by redefining our original IP to see what constraints we need to add. \
We've added some more variables to our IP: 
- $x_j$ is the decision variable that determines whether constructor $j$ is included in the team or not 
- $p_j$ is the # of points that constructor $j$ is worth
- $c_j$ is the cost of constructor $j$

Our budget has also been increased back to **100M**. 


$\text{maximize} \quad \sum_{i=1}^{5} x_i p_i + \sum_{j=1}^{2} x_j p_j$

$\text{subject to} \quad \sum_{i=1}^5 x_i c_i + \sum_{j=1}^2 x_j c_j\leq 100$

$\quad \quad \quad \quad \quad \quad x_i \in (0,1)$ 
$\quad \quad \quad \quad \quad \quad x_j \in (0,1)$ 





## Constraints 

To redefine our greedy algorithm to include constructors, we have to also write helpers that correspond to our new constraints.

In [56]:
budget = 100 # in millions

def under_max_cost_both(items):
    """
    Takes in the picked drivers and constructors, and outputs a bool determining whether 
    it satisfies the first constraint, if it is under the maximum cost

    args:
        drivers: list of strings of the drivers picked in the solution
        constructors: list of strings of the constructors picked in the solution
    returns:
        bool depending on whether the constraint is satisfied
    """
    drivers = list(driver_info.keys())
    constructors = list(constructors_info.keys())
    cum_cost = 0.0 #cumulitive cost of all the drivers
    for item in items:
        # print(driver_info[driver][1])
        if item in drivers:
            cum_cost += float(driver_info[item][1])
        if item in constructors:
            cum_cost += float(constructors_info[item][1])
        # print(f"{cum_cost=}")
    return cum_cost <= budget

## Greedy Algorithm

Update the greedy algorithm to calculate for buying both constructors and drivers. \
Remember, your list of non-increasing value-to-cost ratios should be a combination of both drivers & constructors. 

In [58]:

def greedy_f1(driver_info=driver_info, constructors_info=constructors_info):
    # Replace with your code
    """
    Performs a redux greedy algorithm on all the drivers to pick
    which ones are optimal for the picks

    args:
        driver_info: dict, keys:str, vals: list of str: all the driver info from the csv
    returns:
        selected drivers: list strings of drivers picked
    """

    drivers = list(driver_info.keys())
    constructors = list(constructors_info.keys())

    all_info = driver_info | constructors_info
    sorted_info = sort_items(all_info)

    num_drivers = 0
    num_constructors = 0
    all_selected = []

    for item in sorted_info.keys():
        next_item = []
        next_item = all_selected.copy()
        next_item.append(item)
        if not under_max_cost_both(next_item):
            if item not in all_selected:
                if num_drivers == 5 and item in drivers:
                    pass
                elif num_constructors == 2 and item in constructors:
                    pass
                else:
                    all_selected = compare_items(item, all_selected, all_info)
                    if len(all_selected) == 1:
                        if item in drivers: num_drivers += 1
                        if item in constructors: num_constructors += 1
                            
        else:
            if item not in all_selected:
                if num_drivers == 5 and item in drivers:
                    pass
                elif num_constructors == 2 and item in constructors:
                    pass
                else:
                    all_selected.append(item)
                    if item in drivers: num_drivers += 1
                    if item in constructors: num_constructors += 1
    return all_selected


def compare_items(item, rest_of_items, info):
    """
    Decides which of the 2 drivers inputted are better to be picked. 

    args:
        driver1: string of the first driver
        driver2: string of the second driver
        driver_info: dict, keys:str, vals: list of str: all the driver info from the csv
    returns:
        better_driver: list of strings of the better of the 2 options
    """
    ratios = value_to_cost(info)
    item_ratio = ratios[item]
    rest_ratio = 0.0
    for i in rest_of_items:
        rest_ratio += ratios[i]
    rest_ratio = rest_ratio / len(rest_of_items)
    if rest_ratio >= item_ratio:
        return rest_of_items
    else:
        return [item]
    
gf1 = greedy_f1()
print(gf1)

['Haas', 'Mercedes', 'Esteban Ocon', 'Lance Stroll', 'Lando Norris', 'Oliver Bearman', 'Nico Hulkenberg']
